# ДЗ № 15, Волжина Лена

Реализуйте алгоритм Нусинов для выравнивания РНК. [Задание](https://compscicenter.ru/learning/assignments/27973/)

In [1]:
from collections import defaultdict

DEBUG = False
pair = {'A': 'U', 'G': 'C', 'U': 'A', 'C': 'G'}

![Algorithm](hw15_algo.png)

In [2]:
class Source(object):
    def __init__(self, t, p1, p2=None):
        self.type = t
        self.value = (p1, p2) if p2 is not None else (p1, )
    
    def __repr__(self):
        return '{} {}'.format(self.type, self.value)

    
def print_weigths(weights, s):
    print("     " + "    ".join(s))
    for i in range(len(s)):
        print(s[i], end="    ")
        print("    ".join(str(weights[i].get(j, ' ')) for j in range(len(s))))


class NussinovAligner(object):
    def __init__(self, min_loop_size=2):
        self.min_loop_size = min_loop_size
    
    def move_forward(self, s):
        n = len(s)
        weights, sources = defaultdict(dict), defaultdict(dict)
        
        # fill several diagonals with zero
        for diag in range(-1, self.min_loop_size):
            for i in range(n - diag):
                weights[i][i + diag] = 0
        
        def get_possible_sources(i, j):
            result = [(weights[i + 1][j], Source('step', (i + 1, j))), 
                      (weights[i][j - 1], Source('step', (i, j - 1)))]
            if s[i] == pair[s[j]]:
                result.append((weights[i + 1][j - 1] + 1, Source('pair', (i + 1, j - 1))))
            result.extend((weights[i][k] + weights[k + 1][j], Source('jumper', (i, k), (k + 1, j)))
                          for k in range(i + 1, j))
            return result
        
        # process rest diagonals
        for diag in range(self.min_loop_size, n):
            for i in range(n - diag):
                j = i + diag
                possible_sources = get_possible_sources(i, j)
                
                max_weight, max_source = max(possible_sources, key=lambda x: x[0])
                weights[i][j], sources[i][j] = max_weight, max_source
        
        if DEBUG:
            print_weigths(weights, s)
        
        return weights, sources
    
    def process(self, s):
        weigths, sources = self.move_forward(s)
        
        pairs, coords = [], {(0, len(s) - 1)}
        while coords:
            i, j = coords.pop()
            source = sources[i].get(j)
            if source is None:
                continue
            
            coords.update(source.value)
            if source.type == 'pair':
                pairs.append((i, j))
            
        return pairs
            

![Example](hw15_example.png)

In [7]:
DEBUG = True

# s = "ACCAAGGGUUGGAAC"  # example for weights
s = "AAACAUGAGGAUUACCCAUGU"
aligner = NussinovAligner(min_loop_size=3)
pairs = aligner.process(s)
for i, j in pairs:
    print(i + 1, j + 1)

     A    A    A    C    A    U    G    A    G    G    A    U    U    A    C    C    C    A    U    G    U
A    0    0    0    0    0    1    1    1    1    1    1    2    3    3    3    3    4    4    5    6    7
A    0    0    0    0    0    1    1    1    1    1    1    2    3    3    3    3    4    4    5    6    7
A         0    0    0    0    1    1    1    1    1    1    2    3    3    3    3    4    4    5    6    7
C              0    0    0    0    1    1    1    1    1    2    2    2    2    3    3    4    5    6    6
A                   0    0    0    0    0    0    0    1    2    2    2    2    2    3    4    5    5    5
U                        0    0    0    0    0    0    1    1    1    2    2    2    3    4    4    4    4
G                             0    0    0    0    0    0    1    1    1    2    2    3    3    3    3    4
A                                  0    0    0    0    0    1    1    1    1    2    2    2    3    3    4
G                                    

In [4]:
pairs

[(2, 20), (3, 19), (4, 18), (5, 17), (6, 16), (8, 15), (9, 14), (11, 13)]

In [5]:
print_weigths(weights, s)

NameError: name 'weights' is not defined